## Eigenvalues

Chapter 7.

The eigenvalue problem is to find all $\lambda$ and $v$ such that $Av=\lambda v$ for a matix $A$.

For hand calculations we usually find the roots of $\det(A-\lambda I)$. An $n$ x $n$ matrix has $n$ eigenvalues, counting double roots twice, etc. We can then form a diagonal matrix $D$ from the eigenvalues and a matrix $V$ with columns equal to the corresponding eigenvectors. Then we can write $AV = VD$. If $V$ is invertible, then $A = VDV^{-1}$. This is called the eigenvalue decomposition (EVD). If it exists, then $A$ is said to be diagonalizable.

If $A$ has $n$ distinct eigenvalues, then it is diagonalizable.

If $S$ is non-singular, then the similiarity transform $B = SAS^{-1}$ yields a matrix $B$ with the same eigenvalues as $A$.



Special terms for matrices with complex entries:

* A adjoint of a matrix $A$ is written $A^*$ and is computed as the complex conjugate of its transpose: $A^* = \bar{A}^T$. In Julia the transpose is computed using the single quotation operator: `A'` and for complex matrices, this is the conjugate transpose.
* A unitary matrix is a complex matrix with orthogonal columns, $A^* = A^{-1}$.

## Power method for finding algorithms

Section 8.2 detour.

Starting from an initial guess $v_0$ we can compute $v_{k+1} = Av_{k}$. If the dominant eigenvalue (largest magnitude) is 1, then this sequence will converge to the corresponding eigenvector. In all other cases, we normalize $v_k$ at each iteration to ensure its norm does not shrink to 0 or grow to infinity.

In [20]:
using LinearAlgebra
function my_power(A; v0 = randn(size(A)[1]), n = 100)
    v = v0
    last_lambda = Inf
    lambda = 1.0
    for i in 1:n
        v = A*v
        lambda = norm(v)
        v = v / lambda
        if abs(last_lambda - lambda ) < 1e-8
            break
        end
        last_lambda = lambda
    end
    (lambda, v)
end

my_power (generic function with 1 method)

In [21]:
A = [ 1 2 5 ; -2 4 0; 1 -1 1]

3×3 Matrix{Int64}:
  1   2  5
 -2   4  0
  1  -1  1

In [3]:
my_power(A, v0 = [1,1,1])

(3.675130889221877, [-0.15318478251892112, -0.9430553688467876, 0.29526427771384745])

In [4]:
my_power(A)

(3.675130888130893, [-0.15318478285523296, -0.9430553688206202, 0.29526427762294427])

In [52]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -2.723173538994717
 -1.2367459745066258
 -1.141965620985496
  1.3085729503845156
  3.0872932176375922
vectors:
5×5 Matrix{Float64}:
 -0.189787  -0.0796575  -0.140791   0.614339  -0.7486
 -0.444266   0.751779    0.407867   0.226045   0.141432
  0.223681  -0.0825073   0.761902  -0.357512  -0.484614
 -0.329002  -0.627285    0.452046   0.385337   0.381368
  0.779957   0.167893    0.170243   0.543316   0.198252

In [40]:
A = rand(3,3)
my_power(A)

(2.1951191595121307, [-0.5724353026452579, -0.554936831756638, -0.603624831369069])

In [41]:
eigen(A)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
3-element Vector{ComplexF64}:
 -0.19573317715844563 - 0.16777135539614818im
 -0.19573317715844563 + 0.16777135539614818im
    2.195119158115406 + 0.0im
vectors:
3×3 Matrix{ComplexF64}:
 -0.634161-0.0im       -0.634161+0.0im       0.572435+0.0im
  0.264289-0.377539im   0.264289+0.377539im  0.554937+0.0im
  0.462434+0.414258im   0.462434-0.414258im  0.603625+0.0im

The convergence of this method depends on the ratio of the two largest eigenvalues.

The method can be adapted (Section 8.3, inverse iteration) to find the smallest eigenvalue.

Demo 7.2.12 shows how the QR factorization and iteration can be used to find eigenvalues and eigenvectors, but does not explain how the method works. See the Wikipedia article on the [Francis algorithm](https://en.wikipedia.org/wiki/QR_algorithm).

In [31]:
function my_qr_eigen(A0)
    A = copy(A0)
    for i in 1:40
      Q, R = qr(A)
      A = R*Q
    end
    sort(diag(A))
end

my_qr_eigen (generic function with 1 method)

The sequence $A_k$ of matrices are all similar (they have the same eigenvalues) since $A_{k+1} = R_kQ_k = Q_k^{-1}Q_kR_kQ_k = Q^{-1}_kAQ_k = Q^T_kAQ_k$

This sequence will (under some conditions) converge to the Shur form of $A$, which is upper triangular. The eigenvalues are then the diagonal entries.

In [42]:
my_qr_eigen(A)

3-element Vector{Float64}:
 -0.2677993726800502
 -0.1236669816368414
  2.1951191581154053

In [43]:
eigen(A)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
3-element Vector{ComplexF64}:
 -0.19573317715844563 - 0.16777135539614818im
 -0.19573317715844563 + 0.16777135539614818im
    2.195119158115406 + 0.0im
vectors:
3×3 Matrix{ComplexF64}:
 -0.634161-0.0im       -0.634161+0.0im       0.572435+0.0im
  0.264289-0.377539im   0.264289+0.377539im  0.554937+0.0im
  0.462434+0.414258im   0.462434-0.414258im  0.603625+0.0im

This implementation of the method does not find complex eigenvalues, so a better test is to generate a matrix with random real eigenvalues. First we compute a random orthogonal matrix and random eigenvalues, then use these data to create an eigenvalue problem.

In [53]:
N = 500
D = diagm( 4 .* randn(N) )
V,R = qr(randn(N,N))    # V is unitary
A = V*D*V'

500×500 Matrix{Float64}:
 -0.510518    0.0935994  -0.111381    …   0.135873     0.26149    -0.154912
  0.0935994  -0.725876    0.252312       -0.202695     0.0640606  -0.0890776
 -0.111381    0.252312   -0.068077       -0.181467    -0.0145493   0.554694
  0.285581   -0.112811    0.259708        0.189115     0.0577399   0.249033
 -0.11818    -0.0727093   0.0981581       0.00488836   0.116182    0.27687
 -0.284567   -0.248323   -0.0072064   …  -0.126586    -0.0797326   0.264791
  0.0341852  -0.156028   -0.33149        -0.0815295    0.0882803   0.0732722
  0.161086   -0.356508   -0.277762       -0.029749     0.164844   -0.122175
  0.157164   -0.157559   -0.388171        0.0389507    0.126992    0.196644
 -0.0633053   0.113972   -0.270249        0.00480063  -0.208088    0.0696424
 -0.0433735  -0.281403    0.128181    …  -0.0486774   -0.15857     0.373611
  0.186999   -0.107517   -0.0520448       0.0519109    0.045272   -0.0226283
  0.118922   -0.0685002   0.418507        0.182975     0.182

In [54]:
D

500×500 Matrix{Float64}:
 -5.69266   0.0      0.0       …  0.0     0.0       0.0      0.0
  0.0      -6.39329  0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.182193     0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0       …  0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0       …  0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  ⋮                            ⋱                             
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0

In [55]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
500-element Vector{Float64}:
 -11.79549761230988
 -10.584042707155241
  -9.978395729661592
  -9.283658213976308
  -9.04422574726648
  -8.551749177424062
  -8.339361064366658
  -7.875747759689917
  -7.7749338750655586
  -7.686598513359954
  -7.679565111785401
  -7.5839803579892155
  -7.563754559267353
   ⋮
   6.839999654523953
   6.8488530972037145
   7.217136726140818
   7.3819697814596585
   7.699373456786094
   8.446061916014242
   8.85061865678645
   9.069286822764857
   9.191640117438665
   9.644380090265196
  11.903126832875207
  14.11741101903025
vectors:
500×500 Matrix{Float64}:
  0.00959945   -0.061371     0.0746262   …  -0.0413902   -0.0293884
 -0.0776378    -0.0826047   -0.0844904      -0.0294693   -0.0188618
 -0.0910258     0.0254842   -0.0351268      -0.0347388    0.0223968
 -0.0174106    -0.0094337   -0.0114329       0.0692491   -0.0454228
 -0.0224525     0.0556563    0.0783134       0.0567108    0.0159788
 

In [56]:
D

500×500 Matrix{Float64}:
 -5.69266   0.0      0.0       …  0.0     0.0       0.0      0.0
  0.0      -6.39329  0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.182193     0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0       …  0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0       …  0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  ⋮                            ⋱                             
  0.0       0.0      0.0          0.0     0.0       0.0      0.0
  0

In [57]:
my_qr_eigen(A)

500-element Vector{Float64}:
 -11.7367009917984
 -10.567321709211539
  -9.980495752826357
  -7.431851096258328
  -7.2738478735975605
  -6.990656273457643
  -6.910857646957828
  -6.844999387190443
  -6.650619493765232
  -6.546665557069242
  -6.428811599152057
  -5.877431621805072
  -5.8700732040595565
   ⋮
   2.9893915350519844
   3.173037722959543
   3.201325931092465
   3.210942091948737
   3.2221244381988585
   3.6492339460654435
   4.054515168784374
   4.734668341445101
   5.635922977437742
   9.198550641254712
  11.844084190475199
  14.117404238826417

A small extension will find the complex eigenvalues. Find each 2x2 block (with non-zero values off the diagonal) in the similar matrix $A_k$ and get each pair of complex conjugate eigenvalues from those blocks.

In [15]:
A = [ -1 2 3 ; 1 -1 2 ; 0 -1 2]

3×3 Matrix{Int64}:
 -1   2  3
  1  -1  2
  0  -1  2

In [58]:
eigen(A).values

500-element Vector{Float64}:
 -11.79549761230988
 -10.584042707155241
  -9.978395729661592
  -9.283658213976308
  -9.04422574726648
  -8.551749177424062
  -8.339361064366658
  -7.875747759689917
  -7.7749338750655586
  -7.686598513359954
  -7.679565111785401
  -7.5839803579892155
  -7.563754559267353
   ⋮
   6.839999654523953
   6.8488530972037145
   7.217136726140818
   7.3819697814596585
   7.699373456786094
   8.446061916014242
   8.85061865678645
   9.069286822764857
   9.191640117438665
   9.644380090265196
  11.903126832875207
  14.11741101903025

In [44]:
for i in 1:40
    Q,R = qr(A)
    A = R*Q
end
A

3×3 Matrix{Float64}:
 2.19512       0.00483386   0.237188
 1.59099e-37  -0.123667    -0.160085
 9.7766e-38    0.20827     -0.267799

In [45]:
eigen(A[2:3,2:3]).values

2-element Vector{ComplexF64}:
 -0.1957331771584458 - 0.16777135539614807im
 -0.1957331771584458 + 0.16777135539614807im

In [46]:
quad_roots(a,b,c) = (-b + sqrt(Complex(b^2 - 4*a*c)))/(2*a)
quad_roots(1, -A[2,2]-A[3,3], A[2,2]*A[3,3] - A[2,3]*A[3,2])

-0.1957331771584458 + 0.1677713553961481im

Ta Da!